In [1]:
import torch

In [6]:
input = torch.ones([2, 2], requires_grad=False) # [1.0, 1.0, 1.0, 1.0]
w1 = torch.tensor(2.0, requires_grad=True)      # [2.0, 2.0, 2.0, 2.0]
w2 = torch.tensor(3.0, requires_grad=True)      # [3.0, 3.0, 3.0, 3.0]
w3 = torch.tensor(4.0, requires_grad=True)      # [4.0, 4.0, 4.0, 4.0]

l1 = input * w1  # l1 = input x w1 = [2.0, 2.0, 2.0, 2.0]
l2 = l1 + w2     # l2 = l1 + w2 = [5.0, 5.0, 5.0, 5.0]
l3 = l1 * w3     # l3 = l1 x w3 = [8.0, 8.0, 8.0, 8.0] 
l4 = l2 * l3     # l4 = l2 x l3 = [40.0, 40.0, 40.0, 40.0] 
loss = l4.mean() # loss = mean(l4) = 40.0   loss.grad
print(loss)



# print(w1.data, w1.grad, w1.grad_fn)
# # tensor(2.) None None

# print(l1.data, l1.grad, l1.grad_fn)
# # tensor([[2., 2.],
# #         [2., 2.]]) None <MulBackward0 object at 0x000001EBE79E6AC8>

# print(loss.data, loss.grad, loss.grad_fn)
# # tensor(40.) None <MeanBackward0 object at 0x000001EBE79D8208>

print("#########反向传播########")
loss.backward()

print(w1.grad, w2.grad, w3.grad)
# tensor(28.) tensor(8.) tensor(10.)
print(l1.grad, l2.grad, l3.grad, l4.grad, loss.grad) # 非叶张量，因此为None
# None None None None None

tensor(40., grad_fn=<MeanBackward0>)
#########反向传播########
None
tensor(28.) tensor(8.) tensor(10.)
None None None None None


d:\software\anaconda\envs\gpt\lib\site-packages\torch\_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten\src\ATen/core/TensorBody.h:417.)
  return self._grad


In [38]:
# 尝试获取非叶张量b的梯度值，结果为None
a = torch.ones([2, 2], requires_grad=True)
# a = torch.ones([2, 2], requires_grad=False)
print("a是否是叶张量",a.is_leaf) # 只要是用户创建的张量，均为叶张量，与是否requires_grad无关


b = 3*a + 2 # b是a的深拷贝，不共享内存，但共享计算图（保留梯度传播关系）。
loss=b.sum()
print("b是否是叶张量",b.is_leaf)
print("b是否需要计算梯度",b.requires_grad)
loss.backward()

print(a.grad_fn, a.grad)
print(b.grad_fn, b.grad)


a是否是叶张量 True
b是否是叶张量 False
b是否需要计算梯度 True
None tensor([[3., 3.],
        [3., 3.]])
<AddBackward0 object at 0x000002260C8826D0> None


d:\software\anaconda\envs\gpt\lib\site-packages\torch\_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten\src\ATen/core/TensorBody.h:417.)
  return self._grad


In [42]:
# 尝试获取非叶张量b的梯度值，但采用了保留梯度的设置，可以获得结果
a = torch.ones([2, 2], requires_grad=True)
# a = torch.ones([2, 2], requires_grad=False)
print("a是否是叶张量",a.is_leaf) # 只要是用户创建的张量，均为叶张量，与是否requires_grad无关


b = 3*a + 2 # b是a的深拷贝，不共享内存，但共享计算图（保留梯度传播关系）。
b.retain_grad() # 务必写在反向传播之前！
loss=b.sum()
print("b是否是叶张量",b.is_leaf)
print("b是否需要计算梯度",b.requires_grad)

loss.backward()


print(a.grad_fn, a.grad)
print(b.grad_fn, b.grad)

a是否是叶张量 True
b是否是叶张量 False
b是否需要计算梯度 True
None tensor([[3., 3.],
        [3., 3.]])
<AddBackward0 object at 0x000002260C8AD5B0> tensor([[1., 1.],
        [1., 1.]])


#### inplace 操作,不更改内存地址，就能实现操作，有利于节省内存。

In [49]:
a = torch.tensor([2,1])
print(a.storage().data_ptr()) # 2362253296896
print(a.exp().storage().data_ptr()) # 2362253305216
print(a.storage().data_ptr()) # 2362253296896

# 情景 1
a = a.exp() # 非in-place操作
print(a) # tensor([7.3891, 2.7183])


print(a.storage().data_ptr()) # 2362253303616
# 情景 2
a[0] = 10 # in-place操作
print(a.storage().data_ptr()) # 2362253303616

2362253296896
2362253305216
2362253296896
tensor([7.3891, 2.7183])
2362253303616
2362253303616


In [56]:
# 对非叶张量，在backward之前进行就地操作。会报错
a = torch.tensor([1.0, 3.0], requires_grad=True)
b = a + 2 # b是一个非叶张量。允许在使用前进行就地操作
print(b._version) # 0
print("b是不是叶张量",b.is_leaf)
print("b是不是需要梯度",b.requires_grad)
b[0] = 1000.0 # 在使用之前进行，一次就地
b[0] = 1 # 在使用之前进行，二次就地
loss = (b * b).mean()
# b[0] = 1000.0
print(b._version) # 1

loss.backward()

0
b是不是叶张量 False
b是不是需要梯度 True
2


In [66]:
# 对叶张量，任何时候不允许进行就地操作。会报错
a = torch.tensor([1.0, 3.0], requires_grad=True)
print(a._version) # 0
print("a是不是叶张量",a.is_leaf)
print("a是不是需要梯度",a.requires_grad)
# a += 1 # 就地操作是非法的
a = a+1 # 非就地操作是允许的
print(a, a.is_leaf) # 变成了非叶张量
print(a._version) # 0

b = a + 2 # b是一个非叶张量。允许在使用前进行就地操作
print(b._version) # 0
print("b是不是叶张量",b.is_leaf)
print("b是不是需要梯度",b.requires_grad)
b[0] = 1000.0 # 在使用之前进行，一次就地
b[0] = 1 # 在使用之前进行，二次就地
loss = (b * b).mean()
print(b._version) # 1

loss.backward()

0
a是不是叶张量 True
a是不是需要梯度 True
tensor([2., 4.], grad_fn=<AddBackward0>) False
0
0
b是不是叶张量 False
b是不是需要梯度 True
2


In [93]:
# 走后门（从内存直接修改法）改变叶张量的方法
# 方法一
a = torch.tensor([10., 5., 2., 3.], requires_grad=True)
print(a, a.is_leaf, id(a),a.storage().data_ptr(),a.requires_grad)

a.data.fill_(10.)
# a.detach().fill_(10.)
# a.detach()
print(a, a.is_leaf, id(a),a.storage().data_ptr(),a.requires_grad)



loss = (a*a).mean()
print(loss)
loss.backward()
print(a.grad)
# tensor([5., 5., 5., 5.])

tensor([10.,  5.,  2.,  3.], requires_grad=True) True 2362444863024 2362253303680 True
tensor([10., 10., 10., 10.], requires_grad=True) True 2362444863024 2362253303680 True
tensor([10., 10., 10., 10.], requires_grad=True) True 2362444863024 True
tensor(100., grad_fn=<MeanBackward0>)
tensor([5., 5., 5., 5.])


In [97]:
# 走后门的方法二：只能用于使用前，一般情况下，是用作推理的
a = torch.tensor([10., 5., 2., 3.], requires_grad=True)
print(a, a.is_leaf, id(a),a.storage().data_ptr(),a.requires_grad)


with torch.no_grad():
    a[:] = 10.
print(a, a.is_leaf, id(a),a.storage().data_ptr(),a.requires_grad)


loss = (a*a).mean()

# 放在这里会in-place报错
# with torch.no_grad():
#     a[:] = 10.
# print(a, a.is_leaf, id(a),a.storage().data_ptr(),a.requires_grad)


loss.backward()
print(a.grad)
# tensor([5., 5., 5., 5.])

tensor([10.,  5.,  2.,  3.], requires_grad=True) True 2362445062448 2362253303872 True
tensor([10., 10., 10., 10.], requires_grad=True) True 2362445062448 2362253303872 True
tensor([5., 5., 5., 5.])


In [131]:
# 知乎网友提问：
a = torch.tensor([1.,2.,3.], requires_grad=True)
print(a, a.is_leaf, id(a),a.storage().data_ptr(),a.requires_grad)
out=a.sigmoid()
print(out, out.is_leaf, id(out),out.storage().data_ptr(),out.requires_grad)
# out=torch.sigmoid(a)
# out=a**2

c=out.detach().clone()
print(out, out.is_leaf, id(out),out.storage().data_ptr(),out.requires_grad)
print(c, c.is_leaf, id(c),c.storage().data_ptr(),c.requires_grad)
c[0]=10
c.requires_grad_() # 11和12行，若顺序调换，则报错
print("out就地操作后")
print(out, out.is_leaf, id(out),out.storage().data_ptr(),out.requires_grad)
print("c就地操作后")
print(c, c.is_leaf, id(c),c.storage().data_ptr(),c.requires_grad)

print(out._version)

loss=out*2
loss2=torch.sigmoid(c*6)

loss.sum().backward()
loss2.sum().backward()
print(a.grad)

print(c.grad,c.grad_fn)
print(c._version)
print(out._version)

tensor([1., 2., 3.], requires_grad=True) True 2362443811072 2362253308032 True
tensor([0.7311, 0.8808, 0.9526], grad_fn=<SigmoidBackward0>) False 2362447369856 2362253309568 True
tensor([0.7311, 0.8808, 0.9526], grad_fn=<SigmoidBackward0>) False 2362447369856 2362253309568 True
tensor([0.7311, 0.8808, 0.9526]) True 2362445409232 2362253308864 False
out就地操作后
tensor([0.7311, 0.8808, 0.9526], grad_fn=<SigmoidBackward0>) False 2362447369856 2362253309568 True
c就地操作后
tensor([10.0000,  0.8808,  0.9526], requires_grad=True) True 2362445409232 2362253308864 True
0
tensor([0.3932, 0.2100, 0.0904])
tensor([0.0000, 0.0301, 0.0196]) None
1
0


In [ ]:
# 钩子用法




